<a href="https://colab.research.google.com/github/miguellucero123/Analisis_est/blob/main/Coolabconexion1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Importamos las bibliotecas necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Ruta al archivo
path = '/content/drive/MyDrive/Analisis_estadistico_data/datos_supermercadousa.csv'

# Leer el archivo en formato de texto bruto
df_raw = pd.read_csv(path, sep=';', encoding='latin1', dtype=str)

# Mostrar las primeras filas del DataFrame en bruto
print("Primeras filas del DataFrame original (en bruto):")
display(df_raw.head())

# Función para convertir correctamente los valores numéricos
def fix_numeric_format(value):
    if not isinstance(value, str):
        return value

    # Si el valor tiene un punto (separador de miles) y una coma (decimales)
    if '.' in value and ',' in value:
        # Formato español: 1.234,56
        return float(value.replace('.', '').replace(',', '.'))

    # Si solo tiene punto (puede ser separador decimal en formato inglés o separador de miles)
    elif '.' in value:
        # Verificar si es un separador de miles (por ejemplo 5.489.715)
        if value.count('.') > 1:
            # Esto es un separador de miles, eliminarlos
            return float(value.replace('.', ''))
        # Si solo hay un punto, lo tratamos como separador decimal
        return float(value)

    # Si solo tiene coma (puede ser separador decimal)
    elif ',' in value:
        return float(value.replace(',', '.'))

    # Si no tiene separadores, intentar convertir directamente
    try:
        return float(value)
    except:
        return value

# Crear un DataFrame temporal para procesar los datos
df_temp = df_raw.copy()

# Procesar las columnas numéricas
columnas_numericas = ['Precio por unidad', 'Canrtidad', 'Impuesto 5%', 'Total',
                     'cogs', 'Margen bruto', 'Ingreso bruto', 'Rating']

for col in columnas_numericas:
    if col in df_temp.columns:
        df_temp[col] = df_temp[col].apply(fix_numeric_format)
        print(f"Columna {col}: Convertida a float")

# Convertir fechas
if 'Fecha' in df_temp.columns:
    df_temp['Fecha'] = pd.to_datetime(df_temp['Fecha'], format='%d/%m/%Y', errors='coerce')
    print("Columna Fecha: Convertida a datetime")

if 'Tiempo' in df_temp.columns:
    try:
        df_temp['Tiempo'] = pd.to_datetime(df_temp['Tiempo'], format='%H:%M').dt.time
        print("Columna Tiempo: Convertida a time")
    except:
        print("Columna Tiempo: Se mantiene como string")

# Calcular columnas correctas
df_temp['Impuesto_Calculado'] = df_temp['cogs'] * 0.05
df_temp['Total_Calculado'] = df_temp['cogs'] + df_temp['Impuesto_Calculado']
df_temp['Ingreso_bruto_Calculado'] = df_temp['cogs'] * 0.05

# Verificar los cálculos para la primera fila
primera_fila = df_temp.iloc[0]
print("\nVerificación de cálculos para la primera fila:")
print(f"Precio por unidad: {primera_fila['Precio por unidad']}")
print(f"Cantidad: {primera_fila['Canrtidad']}")
print(f"Valor base (cogs): {primera_fila['cogs']}")
print(f"Impuesto 5% (original): {primera_fila['Impuesto 5%']}")
print(f"Impuesto 5% (calculado): {primera_fila['Impuesto_Calculado']}")
print(f"Total (original): {primera_fila['Total']}")
print(f"Total (calculado): {primera_fila['Total_Calculado']}")
print(f"Ingreso bruto (original): {primera_fila['Ingreso bruto']}")
print(f"Ingreso bruto (calculado): {primera_fila['Ingreso_bruto_Calculado']}")

# CREACIÓN DEL DATAFRAME LIMPIO
# Primero, identificamos las columnas que queremos mantener tal cual
columnas_a_mantener = [col for col in df_temp.columns if col not in
                       ['Impuesto 5%', 'Total', 'Ingreso bruto',
                        'Impuesto_Calculado', 'Total_Calculado', 'Ingreso_bruto_Calculado']]

In [ ]:
# 1. Examinemos ejemplos de las fechas problemáticas
mask_fechas_invalidas = df_temp['Fecha'].isna()
fechas_problematicas = df_raw.loc[mask_fechas_invalidas, 'Fecha'].head(20)
print("Ejemplos de fechas problemáticas:")
print(fechas_problematicas.tolist())

# 2. Contar los diferentes formatos y patrones
print("\nPatrones de fechas en los registros no convertidos:")
patron_fechas = df_raw.loc[mask_fechas_invalidas, 'Fecha'].value_counts().head(10)
print(patron_fechas)

# 3. Solución mejorada para la conversión de fechas
def parse_fecha_mejorado(fecha_str):
    if not isinstance(fecha_str, str) or pd.isna(fecha_str) or fecha_str.strip() == '':
        return pd.NaT

    fecha_str = fecha_str.strip()

    # Lista de formatos a intentar
    formatos = [
        '%d/%m/%Y',  # Día/Mes/Año (España, Latinoamérica)
        '%m/%d/%Y',  # Mes/Día/Año (EEUU)
        '%Y/%m/%d',  # Año/Mes/Día (ISO)
        '%d-%m-%Y',  # Con guiones
        '%m-%d-%Y',
        '%Y-%m-%d',
        '%d.%m.%Y',  # Con puntos
        '%m.%d.%Y',
        '%Y.%m.%d',
        '%b %d, %Y',  # Ej: Jan 21, 2023
        '%d %b %Y',   # Ej: 21 Jan 2023
        '%B %d, %Y',  # Ej: January 21, 2023
        '%d %B %Y'    # Ej: 21 January 2023
    ]

    # Intentar cada formato
    for formato in formatos:
        try:
            return pd.to_datetime(fecha_str, format=formato)
        except:
            continue

    # Si ninguno de los formatos específicos funcionó, intentar con dayfirst=True y False
    try:
        return pd.to_datetime(fecha_str, dayfirst=True, errors='raise')
    except:
        try:
            return pd.to_datetime(fecha_str, dayfirst=False, errors='raise')
        except:
            # Como último recurso, intentar extraer números y crear una fecha
            try:
                # Extraer números de la cadena
                import re
                numeros = re.findall(r'\d+', fecha_str)
                if len(numeros) >= 3:
                    # Asumir que los números son día, mes, año (en ese orden)
                    dia = int(numeros[0])
                    mes = int(numeros[1])
                    anio = int(numeros[2])

                    # Validar los valores
                    if 1 <= dia <= 31 and 1 <= mes <= 12:
                        # Si el año tiene 2 dígitos, convertir a 4 dígitos
                        if anio < 100:
                            anio = 2000 + anio if anio < 50 else 1900 + anio

                        return pd.Timestamp(year=anio, month=mes, day=dia)
            except:
                pass

            return pd.NaT

# 4. Aplicar la nueva función a las fechas
df_temp['Fecha_original'] = df_temp['Fecha']  # Guardar la versión actual por si acaso
df_temp['Fecha'] = df_raw['Fecha'].apply(parse_fecha_mejorado)

# 5. Verificar resultados
fechas_invalidas_nuevas = df_temp['Fecha'].isna().sum()
print(f"\nDespués de la corrección: {fechas_invalidas_nuevas} fechas siguen sin convertirse")

# 6. Si aún hay fechas no convertidas, mostrarlas para análisis manual
if fechas_invalidas_nuevas > 0:
    print("\nEjemplos de fechas que siguen sin poder convertirse:")
    fechas_aun_problematicas = df_raw.loc[df_temp['Fecha'].isna(), 'Fecha'].value_counts().head(10)
    print(fechas_aun_problematicas)

    # 7. Última opción: reemplazar fechas no válidas con un valor predeterminado o la mediana
    fecha_mediana = df_temp['Fecha'].median()
    print(f"\nReemplazar fechas no válidas con la fecha mediana: {fecha_mediana}")
    df_temp['Fecha'] = df_temp['Fecha'].fillna(fecha_mediana)

In [ ]:
df_temp.head()

In [ ]:
# Verificar que todas las fechas convertidas sean del mismo año
print("Distribución de años en las fechas:")
print(df_temp['Fecha'].dt.year.value_counts())

# Visualizar la distribución de fechas
plt.figure(figsize=(12, 5))
df_temp['Fecha'].dt.month.value_counts().sort_index().plot(kind='bar')
plt.title('Distribución de fechas por mes')
plt.xlabel('Mes')
plt.ylabel('Cantidad de registros')
plt.xticks(range(12), ['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic'])
plt.show()

In [ ]:
# Verificar los valores negativos en la columna Total
negativos_count = (df_temp['Total'] < 0).sum()
print(f"Hay {negativos_count} valores negativos en la columna 'Total'")

if negativos_count > 0:
    # Mostrar ejemplos de registros con total negativo
    print("Ejemplos de registros con Total negativo:")
    display(df_temp[df_temp['Total'] < 0].head())

    # Corregir tomando el valor absoluto
    df_temp['Total_corregido'] = df_temp['Total'].abs()

    # O utilizando el valor calculado (si prefieres esta opción)
    # df_temp['Total'] = df_temp['Total_Calculado']

In [ ]:
df_temp.head()

In [ ]:
# Verificar que realmente tenemos las fechas convertidas en la columna Fecha
print("Primeras 5 fechas convertidas:")
print(df_temp['Fecha'].head())

# Mostrar el DataFrame completo con las fechas correctamente convertidas
cols_to_display = ['Invoice ID', 'Sucursal', 'Ciudad', 'Tipo de comprador', 'Género', 'Linea de producto', 'Fecha', 'Tiempo', 'Precio por unidad' ,'Canrtidad', 'Total_Calculado', 'Impuesto_Calculado', 'cogs', 'Ingreso_bruto_Calculado', 'Rating']
display(df_temp[cols_to_display].head())

# Si las fechas están correctamente convertidas, deberíamos poder extraer componentes
print("\nDistribución por mes:")
print(df_temp['Fecha'].dt.month.value_counts().sort_index())

# Nueva sección

In [ ]:
# Crear el DataFrame limpio
df_clean = df_temp[columnas_a_mantener].copy()

# Agregar las columnas calculadas con nombres limpios
df_clean['Impuesto'] = df_temp['Impuesto_Calculado']
df_clean['Total'] = df_temp['Total_Calculado']
df_clean['Ingreso_bruto'] = df_temp['Ingreso_bruto_Calculado']

In [ ]:
# Después de crear df_clean y antes del análisis estadístico:

#=============================================
# LIMPIEZA DE DATOS
#=============================================
print("\n=========== LIMPIEZA DE DATOS ===========")

# 1. Verificar valores nulos
print("\n1. Verificando valores nulos:")
valores_nulos = df_clean.isnull().sum()
print(valores_nulos)

if valores_nulos.sum() > 0:
    print(f"Se encontraron {valores_nulos.sum()} valores nulos en total.")

    # Verificar qué columnas tienen valores nulos
    columnas_con_nulos = valores_nulos[valores_nulos > 0].index.tolist()
    print(f"Columnas con valores nulos: {columnas_con_nulos}")

    # Eliminar filas con valores nulos
    filas_antes = len(df_clean)
    df_clean = df_clean.dropna()
    filas_eliminadas = filas_antes - len(df_clean)
    print(f"Se eliminaron {filas_eliminadas} filas con valores nulos.")
    print(f"Filas restantes: {len(df_clean)}")
else:
    print("No se encontraron valores nulos en el DataFrame.")

# 2. Verificar y eliminar duplicados
print("\n2. Verificando duplicados:")
duplicados = df_clean.duplicated().sum()
print(f"Se encontraron {duplicados} filas duplicadas.")

if duplicados > 0:
    # Mostrar un ejemplo de fila duplicada si existe
    if duplicados > 0:
        print("\nEjemplo de fila duplicada:")
        display(df_clean[df_clean.duplicated(keep='first')].head(1))

    # Eliminar duplicados
    filas_antes = len(df_clean)
    df_clean = df_clean.drop_duplicates()
    filas_eliminadas = filas_antes - len(df_clean)
    print(f"Se eliminaron {filas_eliminadas} filas duplicadas.")
    print(f"Filas restantes: {len(df_clean)}")

# 3. Resumen después de la limpieza
print("\n3. Resumen después de la limpieza:")
print(f"Dimensiones del DataFrame limpio: {df_clean.shape}")
print(f"Filas: {df_clean.shape[0]}, Columnas: {df_clean.shape[1]}")

# Verificar que no queden valores nulos ni duplicados
nulos_restantes = df_clean.isnull().sum().sum()
duplicados_restantes = df_clean.duplicated().sum()

print(f"Valores nulos restantes: {nulos_restantes}")
print(f"Filas duplicadas restantes: {duplicados_restantes}")

# Mostrar las primeras filas del DataFrame limpio
print("\nPrimeras filas del DataFrame limpio después de eliminar nulos y duplicados:")
display(df_clean.head())

In [ ]:
df_clean = df_clean.rename(columns={'Canrtidad': 'Cantidad', 'Margen bruto': 'Margen_bruto', 'Ingreso bruto': 'Ingreso_bruto',
                                    'Tiempo': 'Hora', 'Rating': 'Calificacion', 'Linea de producto': 'Linea_de_producto',
                                    'Precio por unidad': 'Precio_por_unidad', 'Impuesto 5%': 'Impuesto', 'Total': 'Total',
                                    'cogs': 'Coste_de_produccion','Invoice ID': 'Factura_ID'})
# Verificar el DataFrame limpio
print("\nEstructura del DataFrame limpio:")
print(df_clean.info())

print("\nPrimeras filas del DataFrame limpio:")
display(df_clean.head())

# Verificar los datos numéricos en el DataFrame limpio
print("\nEstadísticas descriptivas de las columnas numéricas del DataFrame limpio:")
display(df_clean.describe())

In [ ]:
# 1. Análisis exploratorio de las columnas Impuesto y Total
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Estadísticas descriptivas
print("Estadísticas de la columna Impuesto:")
print(df_temp['Impuesto 5%'].describe())

print("\nEstadísticas de la columna Total:")
print(df_temp['Total'].describe())

# 2. Visualización de la distribución y outliers
plt.figure(figsize=(15, 6))

# Boxplot para Impuesto
plt.subplot(1, 2, 1)
sns.boxplot(y=df_temp['Impuesto 5%'])
plt.title('Boxplot de Impuesto 5%')
plt.ylabel('Valor')

# Boxplot para Total
plt.subplot(1, 2, 2)
sns.boxplot(y=df_temp['Total'])
plt.title('Boxplot de Total')
plt.ylabel('Valor')

plt.tight_layout()
plt.show()

# 3. Identificación de outliers usando el método IQR
def detectar_outliers_iqr(df, columna, factor=1.5):
    Q1 = df[columna].quantile(0.25)
    Q3 = df[columna].quantile(0.75)
    IQR = Q3 - Q1

    limite_inferior = Q1 - factor * IQR
    limite_superior = Q3 + factor * IQR

    outliers = df[(df[columna] < limite_inferior) | (df[columna] > limite_superior)]
    return outliers, limite_inferior, limite_superior

# Detectar outliers en Impuesto
outliers_impuesto, lower_imp, upper_imp = detectar_outliers_iqr(df_temp, 'Impuesto 5%')
print(f"\nOutliers en Impuesto 5%: {len(outliers_impuesto)} registros")
print(f"Límites para Impuesto 5%: [{lower_imp:.2f}, {upper_imp:.2f}]")

if len(outliers_impuesto) > 0:
    print("Ejemplos de outliers en Impuesto 5%:")
    display(outliers_impuesto[['cogs', 'Impuesto 5%', 'Impuesto_Calculado', 'Total']].head())

# Detectar outliers en Total
outliers_total, lower_total, upper_total = detectar_outliers_iqr(df_temp, 'Total')
print(f"\nOutliers en Total: {len(outliers_total)} registros")
print(f"Límites para Total: [{lower_total:.2f}, {upper_total:.2f}]")

if len(outliers_total) > 0:
    print("Ejemplos de outliers en Total:")
    display(outliers_total[['cogs', 'Impuesto 5%', 'Total', 'Total_Calculado']].head())

# 4. Verificar si los outliers son errores comparando con valores calculados
# Comprobando discrepancias entre valores originales y calculados
df_temp['Error_Impuesto'] = abs(df_temp['Impuesto 5%'] - df_temp['Impuesto_Calculado'])
df_temp['Error_Total'] = abs(df_temp['Total'] - df_temp['Total_Calculado'])

# Identificar registros con grandes discrepancias
umbral_error = 0.1  # 10% de diferencia
errores_impuesto = df_temp[df_temp['Error_Impuesto'] > umbral_error * df_temp['Impuesto_Calculado']]
errores_total = df_temp[df_temp['Error_Total'] > umbral_error * df_temp['Total_Calculado']]

print(f"\nRegistros con errores significativos en Impuesto: {len(errores_impuesto)}")
print(f"Registros con errores significativos en Total: {len(errores_total)}")

# 5. Corrección de outliers y errores
# Crear copia para preservar datos originales
df_corregido = df_temp.copy()

# Método 1: Reemplazar outliers con el valor calculado correcto
# Para Impuesto
mask_outliers_impuesto = (df_corregido['Impuesto 5%'] < lower_imp) | (df_corregido['Impuesto 5%'] > upper_imp)
df_corregido.loc[mask_outliers_impuesto, 'Impuesto 5%'] = df_corregido.loc[mask_outliers_impuesto, 'Impuesto_Calculado']

# Para Total
mask_outliers_total = (df_corregido['Total'] < lower_total) | (df_corregido['Total'] > upper_total)
df_corregido.loc[mask_outliers_total, 'Total'] = df_corregido.loc[mask_outliers_total, 'Total_Calculado']

# Método 2: Recalcular Impuesto y Total para todos los registros basados en cogs
df_corregido['Impuesto_Corregido'] = df_corregido['cogs'] * 0.05
df_corregido['Total_Corregido'] = df_corregido['cogs'] + df_corregido['Impuesto_Corregido']

# 6. Verificar resultados después de la corrección
print("\nEstadísticas después de la corrección (Método 1 - Reemplazo de outliers):")
print("Impuesto 5% corregido:")
print(df_corregido['Impuesto 5%'].describe())
print("\nTotal corregido:")
print(df_corregido['Total'].describe())

print("\nEstadísticas usando valores recalculados (Método 2 - Recálculo completo):")
print("Impuesto recalculado:")
print(df_corregido['Impuesto_Corregido'].describe())
print("\nTotal recalculado:")
print(df_corregido['Total_Corregido'].describe())

# 7. Visualizar los resultados después de la corrección
plt.figure(figsize=(15, 10))

# Impuesto: antes y después de la corrección
plt.subplot(2, 2, 1)
sns.boxplot(y=df_temp['Impuesto 5%'])
plt.title('Impuesto 5% (Original)')
plt.ylabel('Valor')

plt.subplot(2, 2, 2)
sns.boxplot(y=df_corregido['Impuesto_Corregido'])
plt.title('Impuesto 5% (Corregido)')
plt.ylabel('Valor')

# Total: antes y después de la corrección
plt.subplot(2, 2, 3)
sns.boxplot(y=df_temp['Total'])
plt.title('Total (Original)')
plt.ylabel('Valor')

plt.subplot(2, 2, 4)
sns.boxplot(y=df_corregido['Total_Corregido'])
plt.title('Total (Corregido)')
plt.ylabel('Valor')

plt.tight_layout()
plt.show()

# 8. Crear el DataFrame final limpio
df_limpio = df_corregido.copy()

# Decidir qué versión de las columnas usar (puedes ajustar según los resultados)
df_limpio['Impuesto'] = df_limpio['Impuesto_Corregido']  # Usar el valor recalculado
df_limpio['Total'] = df_limpio['Total_Corregido']        # Usar el valor recalculado

# Eliminar columnas temporales o redundantes
columnas_a_eliminar = ['Impuesto 5%', 'Error_Impuesto', 'Error_Total',
                       'Impuesto_Calculado', 'Total_Calculado', 'Ingreso_bruto_Calculado',
                       'Impuesto_Corregido', 'Total_Corregido']

df_limpio = df_limpio.drop(columns=[col for col in columnas_a_eliminar if col in df_limpio.columns])

# Mostrar el DataFrame limpio final
print("\nDataFrame limpio final (primeras filas):")
display(df_limpio.head())

In [ ]:
print("\n\nREALIZANDO ANÁLISIS ESTADÍSTICO CON EL DATAFRAME df_clean:")
print("=========================================================")

# 1) Media de la columna de Impuesto
media_impuesto = df_clean['Impuesto'].mean()

print(f"\n1) Media de la columna 'Impuesto': {media_impuesto:.4f}")
print("\nExplicación:")
print("La media aritmética se calcula sumando todos los valores de la columna 'Impuesto'")
print("y dividiendo el resultado entre el número total de datos.")
print(f"Suma de todos los impuestos: {df_clean['Impuesto'].sum():.4f}")
print(f"Número de datos: {len(df_clean['Impuesto'])}")
print(f"Media = Suma / Número de datos = {df_clean['Impuesto'].sum():.4f} / {len(df_clean['Impuesto'])} = {media_impuesto:.4f}")

In [ ]:
# 2) Mediana de la columna Impuesto
mediana_impuesto = df_clean['Impuesto'].median()

print(f"\n2) Mediana de la columna 'Impuesto': {mediana_impuesto:.4f}")
print("\nSentencias para obtener el 50% de los datos ordenados:")
print("df_clean['Impuesto'].median()")
print("# O también:")
print("df_clean['Impuesto'].quantile(0.5)")
print("# O también usando numpy:")
print("np.median(df_clean['Impuesto'])")

In [ ]:
# 3) Moda de la Línea de Producto
moda_linea = df_clean['Linea_de_producto'].mode()[0]
conteo_moda = df_clean['Linea_de_producto'].value_counts().iloc[0]

print(f"\n3) Moda de 'Linea_de_producto': {moda_linea}")
print(f"Esta línea aparece {conteo_moda} veces en el dataset.")

In [ ]:
# 4) Tercer cuartil (75%) de la columna Total
q3_total = df_clean['Total'].quantile(0.75)

#print(f"\n4) ¿Cuál es el valor que corta el 75% de los datos en la columna 'Total' ordenados de menor a mayor?")
print(f"El valor que corta el 75% de los datos (tercer cuartil) es: {q3_total:.4f}")
print("\nExplicación:")
print("Para encontrar este valor, primero ordenamos todos los datos de la columna 'Total' de menor a mayor.")
print("Luego identificamos el valor por debajo del cual se encuentra el 75% de los datos.")
print("Este valor es conocido como el tercer cuartil (Q3) o el percentil 75.")
print("\nPara calcularlo, usamos la función quantile() con el parámetro 0.75:")
print("df_clean['Total'].quantile(0.75)")

# Valores ordenados para verificar
total_ordenado = sorted(df_clean['Total'])
print(f"\nTotal de valores en la columna 'Total': {len(total_ordenado)}")
print(f"Posición del valor que corta el 75% de los datos: {int(len(total_ordenado) * 0.75)}")
position_75 = int(len(total_ordenado) * 0.75)
print(f"Valor en esa posición: {total_ordenado[position_75-1]}")

In [ ]:
# 5) Rango intercuartil de Ingreso Bruto
q1_ingreso = df_clean['Ingreso_bruto'].quantile(0.25)
q3_ingreso = df_clean['Ingreso_bruto'].quantile(0.75)
iqr_ingreso = q3_ingreso - q1_ingreso

print(f"\n5) Rango intercuartil (IQR) de 'Ingreso_bruto': {iqr_ingreso:.4f}")
print("\nExplicación:")
print(f"Primer cuartil (Q1, 25%): {q1_ingreso:.4f}")
print(f"Tercer cuartil (Q3, 75%): {q3_ingreso:.4f}")
print(f"Rango intercuartil (IQR) = Q3 - Q1 = {q3_ingreso:.4f} - {q1_ingreso:.4f} = {iqr_ingreso:.4f}")
print("El IQR representa la dispersión del 50% central de los datos.")

In [ ]:
# 6) Varianza de la variable edad (calculada desde la fecha)
fecha_referencia = pd.to_datetime('01/01/2019', format='%d/%m/%Y')
df_clean['Edad_Dias'] = (df_clean['Fecha'] - fecha_referencia).dt.days

# Filtramos para el primer trimestre de 2019 (enero a marzo)
df_q1_2019 = df_clean[(df_clean['Fecha'] >= '2019-01-01') & (df_clean['Fecha'] <= '2019-03-31')]

# Filtramos para todo el año 2019
df_2019 = df_clean[df_clean['Fecha'].dt.year == 2019]

# Calculamos varianzas
varianza_q1_2019 = df_q1_2019['Edad_Dias'].var()
varianza_2019 = df_2019['Edad_Dias'].var()

print(f"\n6) Varianza de la edad en días:")
print(f"a) Para el primer trimestre de 2019: {varianza_q1_2019:.4f}")
print(f"b) Para todo el año 2019: {varianza_2019:.4f}")
print("\nExplicación:")
print("La varianza mide la dispersión de los datos respecto a la media.")
print("Se calcula sumando los cuadrados de las diferencias entre cada valor y la media,")
print("y dividiendo el resultado entre el número de datos menos 1 (corrección de Bessel).")
print(f"\nPrimer trimestre 2019:")
print(f"Media de edad en días: {df_q1_2019['Edad_Dias'].mean():.4f}")
print(f"Número de registros: {len(df_q1_2019)}")
print(f"Varianza = Σ(xi - media)² / (n-1) = {varianza_q1_2019:.4f}")

In [ ]:
# 7) Desviación estándar de Ingreso Bruto
desv_est_ingreso = df_clean['Ingreso_bruto'].std()

print(f"\n7) Desviación estándar de 'Ingreso_bruto': {desv_est_ingreso:.4f}")
print("\nExplicación:")
print("La desviación estándar es la raíz cuadrada de la varianza.")
print("Mide la dispersión promedio de los valores respecto a la media.")
print(f"Varianza de Ingreso_bruto: {df_clean['Ingreso_bruto'].var():.4f}")
print(f"Desviación estándar = √Varianza = √{df_clean['Ingreso_bruto'].var():.4f} = {desv_est_ingreso:.4f}")

In [ ]:
# Resumen final
print("\n\nRESUMEN DE RESULTADOS DEL ANÁLISIS:")
print("===================================")
print(f"1. Media de la columna 'Impuesto': {media_impuesto:.4f}")
print(f"2. Mediana de la columna 'Impuesto': {mediana_impuesto:.4f}")
print(f"3. Moda de 'Linea de producto': {moda_linea}")
print(f"4. Tercer cuartil (75%) de 'Total': {q3_total:.4f}")
print(f"5. Rango intercuartil de 'Ingreso_bruto': {iqr_ingreso:.4f}")
print(f"6a. Varianza de edad (días) para Q1 2019: {varianza_q1_2019:.4f}")
print(f"6b. Varianza de edad (días) para todo 2019: {varianza_2019:.4f}")
print(f"7. Desviación estándar de 'Ingreso_bruto': {desv_est_ingreso:.4f}")

In [ ]:
# Gráficos para visualizar los resultados
plt.figure(figsize=(20, 15))

# 1. Histograma y media del impuesto
plt.subplot(3, 3, 1)
sns.histplot(df_clean['Impuesto'], kde=True)
plt.axvline(media_impuesto, color='r', linestyle='--', label=f'Media: {media_impuesto:.2f}')
plt.axvline(mediana_impuesto, color='g', linestyle='--', label=f'Mediana: {mediana_impuesto:.2f}')
plt.title('Distribución del Impuesto')
plt.legend()

# 2. Mediana del impuesto (boxplot)
plt.subplot(3, 3, 2)
sns.boxplot(x=df_clean['Impuesto'])
plt.title('Boxplot del Impuesto')
plt.xlabel('Impuesto')

# 3. Moda de Línea de producto
plt.subplot(3, 3, 3)
top_lineas = df_clean['Linea_de_producto'].value_counts().head(10)
sns.barplot(y=top_lineas.index, x=top_lineas.values)
plt.title('Top 10 Líneas de producto')
plt.xlabel('Frecuencia')
plt.ylabel('Línea_de_producto')

# 4. Tercer cuartil del Total
plt.subplot(3, 3, 4)
sns.histplot(df_clean['Total'], kde=True)
plt.axvline(q3_total, color='r', linestyle='--', label=f'Q3: {q3_total:.2f}')
plt.title('Distribución del Total con Q3')
plt.legend()

# 5. Rango intercuartil del Ingreso bruto
plt.subplot(3, 3, 5)
sns.boxplot(x=df_clean['Ingreso_bruto'])
plt.title('Boxplot del Ingreso bruto')
plt.xlabel('Ingreso bruto')

# 6. Distribución de Edad_Dias para Q1 2019
plt.subplot(3, 3, 6)
if len(df_q1_2019) > 0:
    sns.histplot(df_q1_2019['Edad_Dias'], kde=True)
    plt.title('Distribución de Edad en días (Q1 2019)')
else:
    plt.text(0.5, 0.5, "No hay datos para Q1 2019", horizontalalignment='center')
    plt.title('Q1 2019 - Sin datos')

# 7. Desviación estándar del Ingreso bruto
plt.subplot(3, 3, 7)
sns.histplot(df_clean['Ingreso_bruto'], kde=True)
media_ingreso = df_clean['Ingreso_bruto'].mean()
plt.axvline(media_ingreso, color='r', linestyle='--', label=f'Media: {media_ingreso:.2f}')
plt.axvline(media_ingreso + desv_est_ingreso, color='g', linestyle='--',
            label=f'Media + DE: {media_ingreso + desv_est_ingreso:.2f}')
plt.axvline(media_ingreso - desv_est_ingreso, color='g', linestyle='--',
            label=f'Media - DE: {media_ingreso - desv_est_ingreso:.2f}')
plt.title('Distribución del Ingreso bruto con DE')
plt.legend()

# Matriz de correlación
plt.subplot(3, 3, 8)
columnas_corr = ['Precio_por_unidad', 'Cantidad', 'Coste_de_produccion', 'Impuesto', 'Total', 'Ingreso_bruto']
corr = df_clean[columnas_corr].corr()
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Matriz de Correlación')

plt.tight_layout()
plt.show()